# Doing Well?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import matplotlib as mpl
from pathlib import Path
from datetime import datetime

# matplotlib default
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('default')

# basic config
Lx, Ly, cut_pos = 4, 4, 2
N = Lx * Ly
delta = 1.0
J0 = 1.0
T_total, nt = 10.0, 200
tlist = np.linspace(0.0, T_total, nt)

# index
idx = lambda x, y: y * Lx + x
# ramp time
t_ramp_time = [0.0, 5.0]

# nn bond
def nn_bonds_4by4():
    B = []
    for y in range(Ly):
        for x in range(Lx):
            if x + 1 < Lx:
                B.append((idx(x,y), idx(x+1, y)))
            if y + 1 < Ly:
                B.append((idx(x, y), idx(x, y+1)))
    return [tuple(sorted(b)) for b in B]

bonds_all = nn_bonds_4by4()

# カット境界
cut_list = [(idx(1, y), idx(2, y)) for y in range(Ly)]
# カット境界に入らないボンドたち
keep_bonds = [b for b in bonds_all if not in cut_list]

# 準備
# スピン演算子
sx, sy, sz = [0.5 * M for M in (qt.sigmax(), qt.sigmay(), qt.sigmaz())]
I2 = qt.qeye(2)

# 指定したサイトのみopを作用させる
def op_at(i, op):
    return qt.tensor([op if k == i else I2 for k in range(N)])

S = [(op_at(i, sx), op_at(i, sy), op_at(i, sz)) for i in range(N)]

def H_bond_unit(i, j, delta=delta):
    Sxi, Syi, Szi = S[i]
    Sxj, Syj, Szj = S[j]
    return (Sxi * Sxj + Syi * Syj + delta * Szi * Szj)

# ハミルトニアン
H_keep_unit = sum(H_bond_unit(i, j) for (i, j) in keep_bonds)
Hm_unit_list = [H_bond_unit(i, j) for (i, j) in cut_list]
H_all_unit = H_keep_unit + sum(Hm_unit_list)